In [6]:
!pip install rasterio scikit-image 

In [7]:
import rasterio
from rasterio.plot import show
from rasterio.features import shapes
import numpy as np
from skimage.segmentation import slic

def crop_distribution_monitoring(image_path):
    # Read the GeoTIFF image
    with rasterio.open(image_path) as src:
        # Read the image as a 3D array (height, width, bands)
        image = src.read()

    # Convert the image from 3D to 2D array (height, width, bands) -> (bands, height*width)
    image_reshaped = image.reshape(image.shape[0], -1)

    # Perform image segmentation using SLIC (Simple Linear Iterative Clustering)
    # Adjust the n_segments parameter based on the expected number of crops in the image
    segments = slic(image_reshaped, n_segments=100, compactness=10, sigma=1)

    # Calculate the area of each segment (crop region) in pixels
    segment_areas = [np.sum(segments == segment_id) for segment_id in np.unique(segments)]

    # Get unique crop types present in the image
    unique_crop_types = np.unique(segments)

    # Calculate the area of each crop type in hectares (assuming pixel size in meters)
    pixel_size_meters = src.res[0]  # Assuming square pixels (x and y resolution are the same)
    pixel_size_hectares = pixel_size_meters * 0.0001  # Convert pixel size to hectares
    crop_type_areas = [area * pixel_size_hectares for area in segment_areas]

    # Display the segmented image (optional)
    show(segments, cmap='tab20', title='Crop Distribution')

    # Print crop type and its corresponding area in hectares
    for i, crop_type in enumerate(unique_crop_types):
        print(f"Crop Type {crop_type}: Area = {crop_type_areas[i]:.2f} hectares")

if __name__ == "__main__":
    # Generate Sattellite Images from Here
    # https://eos.com/blog/free-satellite-imagery-sources/
    image_path = 
    crop_distribution_monitoring(image_path)


ValueError: channel_axis=-1 indicates multichannel, which is not supported for a two-dimensional image; use channel_axis=None if the image is grayscale